In [18]:
import re
import pickle 
import sqlite3 as sl
from unicodedata import normalize
import pandas as pd
from pathlib import Path
import nltk
from nltk.corpus import stopwords
from sklearn.naive_bayes import BernoulliNB
from scipy import sparse
import numpy as np
nltk.download('stopwords')
con = sl.connect('../coleta_de_dados/banco_scraping_olx.db')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ANACLARASAMARINO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
def normaliza_texto_vetorizado(text:str)->str: 
    """
    função criada para transformar os títulos dos anuncios em um formato padronizado, sem caracteres especiais e sem números
    """
    frase_minuscula = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII').lower() #normaliza para retirar acentos e transforma para minuscula
    somente_palavras_validas = re.sub("([^\w]+)|((\w|d)*[\d](\w|d)*)"," ",frase_minuscula) #substitui a string acima por uma string sem numeros e sem caracteres especiais
    texto_normalizado = re.sub("\s+"," ",somente_palavras_validas).strip() #substitui um ou mais espaços da string acima por um único espaço
    return texto_normalizado.split(" ") #retorna uma lista com as palavras válidas 

In [20]:
todos_titulos = pd.read_sql("""
SELECT DISTINCT 
	url_anuncio,	
	titulo_anuncio	
FROM
	anuncios_resumo
""", con, chunksize=1000000)
#pega as urls dos anuncios e os títulos (considerando apenas distintos) no banco de dados. Chuncksize para pegar por partes para não estourar memória 

In [21]:
def constroi_dicionario_palavras(df_pandas_chunked,dicionario):
    """ 
    Função para construir o dicionário com as palavras dos títulos dos anúncios e adicionar a quantidade referente a cada palavra
    """    
    qt_linhas_processadas = 0
    for df in df_pandas_chunked: #para cada subdivisão gerada pelo chunked
        qt_linhas_processadas += len(df) #salva em uma variável a qtd de linhas já processadas
        print(f"qt linhas processadas: {qt_linhas_processadas}") #imprime a qtd de linhas processadas até então
        for titulo in df['titulo_anuncio']: #para cada título nos títulos retornadas, segue
            titulo_vetorizado = normaliza_texto_vetorizado(titulo) #normaliza o título e transforma em uma lista de palavras
            for palavras_titulo in titulo_vetorizado: #para cada palavra no título,
                if palavras_titulo: #se palavras_titulo for válida, segue
                    if dicionario.get(palavras_titulo): #verifica se existe palavras_titulo no dicionário
                        dicionario[palavras_titulo]+=1 #soma 1 unidade ao valor correspondente à chave palavras_titulo
                    else:
                        dicionario[palavras_titulo]=1 #se não tiver a chave no dicionário, adiciona e atribui 1 unidade para ela

In [22]:
def filtrar_dicionario(dicionario_palavras,qtd_minima_palavras_usadas = 5):
    """ 
    Função para filtrar o dicionário retirando todas as preposições e artigos e palavras com menos de 3 letras, e pegando apenas as palavras com mais repetições do que informado como parâmetro
    """ 
    artigos_preposicoes = stopwords.words('portuguese') #pega uma lista de palavras em portugues da biblioteca nltk.corpus
    todas_chaves = dicionario_palavras.keys() #guarda as chaves do dicionário em uma lista
    chaves_para_excluir=[] #lista com as chaves para excluir do dicionário
    for chave in todas_chaves: 
        if chave in artigos_preposicoes or dicionario_palavras[chave] < qtd_minima_palavras_usadas or len(chave)<3:
            chaves_para_excluir.append(chave) #se a palavra for preposição/artigo, ou tiver menos do que o limite determinado no parâmetro ou tiver menos de 3 letras,
    for chave_ex in chaves_para_excluir: #adiciona a palavra na lista para deleção
        del dicionario_palavras[chave_ex] #deleta do dicionário todas as chaves que estão na lista para deleção

In [23]:
dicionario_todas_palavras = {}

if not Path('./dicionario_todas_palavras.pkl').is_file(): #se o caminho indicado não for um arquivo
    constroi_dicionario_palavras(todos_titulos,dicionario_todas_palavras) #constroi o dicionario 
    filtrar_dicionario(dicionario_todas_palavras,30) #filtra o dicionário conforme descrição da função filtrar_dicionario 
    with open('./dicionario_todas_palavras.pkl', 'wb') as file: #cria um arquivo para escrita e abre para escrita
        pickle.dump(dicionario_todas_palavras, file) #coloca a variável(dicionario) no arquivo
else : #se o caminho indicado for um arquivo,
    with open('./dicionario_todas_palavras.pkl', 'rb') as file: #abrir para leitura
        dicionario_todas_palavras = pickle.load(file) #transforma arquivo em variável (dicionário), carrega e salva em dicionario_todas_palavras

lista_todas_palavras = [*dicionario_todas_palavras.keys()] #transforma as chaves do dicionario em lista e salva em lista_todas_palavras     

In [24]:
df_todas_categorias = pd.read_sql("""
SELECT DISTINCT 
	categoria_atual
FROM
	anuncios_resumo
""", con) #pega todas as categorias(finais) do banco de dados
lista_todas_categorias = [*df_todas_categorias['categoria_atual']] #transforma as categorias acima em uma lista e atribui a lista_todas_categorias

In [25]:
def categoria_para_indice(categoria,lista_todas_categorias):
    """verifica qual o índice da categoria informada conforme a lista_todas_categorias"""
    if lista_todas_categorias.count(categoria): #se existir a categoria na lista,
        return lista_todas_categorias.index(categoria) #retorna o indice da categoria na lista_todas_categorias
    else:
        return -1 #caso contrário, retorna -1
        
def titulo_para_indices(titulo,lista_todas_palavras): 
    """verifica qual o indice """
    lista_palavras_titulo = normaliza_texto_vetorizado(titulo) #normaliza o título informado
    indices_titulo =[] #lista de indices (conforme lista_todas_palavras) referentes as palavras do título
    for palavra_titulo in lista_palavras_titulo: #para cada palavra na lista_palavras_titulo,
        if lista_todas_palavras.count(palavra_titulo): #se existir a palavra do título na lista lista_todas_palavras, 
            indices_titulo.append(lista_todas_palavras.index(palavra_titulo)) #adiciona o indice da palavra(indice referente a lista_todas_palavras) na indices_titulo
    return indices_titulo   #retorna uma lista com os indices do titulo              

In [26]:
#testes:
#categoria_para_indice('Artigos infantis',lista_todas_categorias)
#titulo_para_indices('baba de cachorro com casa grande',lista_todas_palavras)

In [27]:
lista_precos = [*range(10)] # coluna que irá classificar o preço (sera usado log 10)

In [28]:
def preco_para_indice(preco,indice_start=0):
    """
    Atribui um índice para um range de preços na escala log
    """
    if isinstance(preco,float) or isinstance(preco,int): #se o preço for int ou float retorna true e segue
        if preco>1:
            index = np.ceil(np.log10(preco)) #arredonda o valor de Log(preço) na base 10 para servir como um índice
            index = int(index) if index>=0 and index<len(lista_precos) else 0 #passa o indice para inteiro se indice for >= 0 e for menor do que o tamaho da lista_preços
            return index+indice_start #indice_start -> começa depois do termino da matriz de palavras
    return 0

In [29]:
def monta_sql_por_categoria(categoria):
    return f"""SELECT DISTINCT 
                    url_anuncio,
                    titulo_anuncio,
                    categoria_atual,
                    preco_anuncio
                FROM
                    anuncios_resumo
                WHERE categoria_atual = '{categoria}'
                --LIMIT 1000 -- Colocar um limit para testar
                """
    #query para pegar url_anuncio, titulo_anuncio, categoria_atual,preco_anuncio referentes a uma categoria específicca

In [30]:
classes_y = [*range(len(lista_todas_categorias))] #cria uma lista com indices de 0 ao tamaho da lista_todas_categorias para ser índice das categorias

In [31]:
clf = BernoulliNB() # Modelo Usado (teste com Bernoulli)

In [32]:
for categoria in lista_todas_categorias: 
    """
    Cria matriz esparsa e treina o modelo com base nela
    """
    df_cat_atual = pd.read_sql(monta_sql_por_categoria(categoria),con)
    print(f'Treinando Modelo para a Categoria: {categoria}')
    qt_colunas = len(lista_todas_palavras)+len(lista_precos) #qtd de colunas da matriz é a soma do tamanho das duas listas
    X_linhas = []
    X_colunas = []
    for linha in range(len(df_cat_atual)): # Adiciona valores na matrix esparsa
        X_colunas_atual = titulo_para_indices(df_cat_atual['titulo_anuncio'][linha],lista_todas_palavras) #adiciona em X_colunas_atual os indices referentes as palavras
        X_colunas_atual += [preco_para_indice(df_cat_atual['preco_anuncio'][linha],len(lista_todas_palavras))] #adiciona em X_colunas_atual o índice referente a coluna de preço
        X_colunas+=X_colunas_atual #adiciona no vetor todos os indices das colunas utilizados
        X_linhas += [linha]*len(X_colunas_atual) #preenche com o valor da linha um vetor do tamanho da coluna atual
    Dados_X = np.full(len(X_linhas),1,dtype=np.int8) #retorna um array do tamanho do x_linhas preenchido com 1
    X = sparse.coo_matrix((Dados_X, (X_linhas, X_colunas)), shape=(len(df_cat_atual), qt_colunas)) #matriz esparsa: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html         
    Y = np.full(len(df_cat_atual),categoria_para_indice(df_cat_atual['categoria_atual'][0],lista_todas_categorias)) #preenche todo vetor Y com o nome da categoria em que irá treinar o modelo 
    clf.partial_fit(X,Y,classes=classes_y) #Treina o Modelo      

Treinando Modelo para a Categoria: Aluguel - casas e apartamentos
Treinando Modelo para a Categoria: Animais para agropecuária
Treinando Modelo para a Categoria: Antiguidades
Treinando Modelo para a Categoria: Aquários e acessórios
Treinando Modelo para a Categoria: Artigos infantis
Treinando Modelo para a Categoria: Barcos e aeronaves
Treinando Modelo para a Categoria: Beleza e saúde
Treinando Modelo para a Categoria: Bijouterias, relógios e acessórios
Treinando Modelo para a Categoria: Bolsas, malas e mochilas
Treinando Modelo para a Categoria: CDs, DVDs etc
Treinando Modelo para a Categoria: Cachorros
Treinando Modelo para a Categoria: Caminhões
Treinando Modelo para a Categoria: Carros, vans e utilitários
Treinando Modelo para a Categoria: Cavalos
Treinando Modelo para a Categoria: Celulares e telefonia
Treinando Modelo para a Categoria: Ciclismo
Treinando Modelo para a Categoria: Computadores e acessórios
Treinando Modelo para a Categoria: Comércio e indústria
Treinando Modelo par

In [33]:
def procura_categoria(titulo,preco=None,qt_impr=5):
    """ 
    Função para testar as entradas com base no título e no preço do produto a ser cadastrado
    """
    qt_colunas = len(lista_todas_palavras)+len(lista_precos) #qt de colunas da matriz
    XN = sparse.dok_matrix((1, qt_colunas), dtype=np.int8) #constrói matriz esparsa XN de forma incremental
    if preco:
        XN[0,preco_para_indice(preco,len(lista_todas_palavras))] = 1 #popula 1 na coluna referente ao preço

    indices_palavras_titulo = titulo_para_indices(titulo,lista_todas_palavras) #lista com os indices das palavras do titulo
    for indice_palavra in indices_palavras_titulo: 
        XN[0,indice_palavra] = 1 #popula 1 nas colunas referentes às palavras do título com base nos índices de indices_palavras_titulo

    categorias_prob = clf.predict_proba(XN[0]) # retorna a probabilidade referente a cada categoria
    indice_categorias_ordenado = np.flip(np.argsort(categorias_prob)) #ordena os indices de acordo com a probabilidade decrescente
    limite_impressoes = qt_impr #variável que define a quantidade de categorias a serem impressas
    for indice_categoria in indice_categorias_ordenado.tolist()[0]:
        print(f'{lista_todas_categorias[indice_categoria]}:{categorias_prob[0][indice_categoria]*100:.2f}%') #imprime uma categoria e o percentual referente a ela (ordem decrescente)
        if limite_impressoes<=1:
            break
        limite_impressoes -=1
        

In [37]:
procura_categoria('blusa')

Roupas e calçados:65.11%
Moda e beleza:28.26%
Artigos infantis:3.92%
Esportes e lazer:0.70%
Beleza e saúde:0.38%


In [35]:
model_add_data = {}
model_add_data['lista_todas_palavras']  = lista_todas_palavras
model_add_data['lista_precos'] = lista_precos
model_add_data['lista_todas_categorias']= lista_todas_categorias

with open('./BernoulliNB_model.pkl', 'wb') as file:
    pickle.dump(clf, file)
with open('./BernoulliNB_model_adds.pkl', 'wb') as file:
    pickle.dump(model_add_data, file)